In [ ]:
from transformers import pipeline
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
#Basic usage
classifier = pipeline("sentiment-analysis")
print(type(classifier))

In [ ]:
print(classifier("I don't know what I am doing."))

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.current_device()

In [ ]:
#Use the GPU
classifier = pipeline("sentiment-analysis", device = "0x000001AC2532E3B0")

In [ ]:
#turn on a dataframe from the .csv file
#view the dataframe
#filter the dataframe using the "airline sentiment" and "text" columns
#plot a histogram to see the data (of the sentiment)
#map the labels to integers (1,0) by creating a dictionary which maps "positive" to 1 and "negative" to "0" and call the map function to add it to df?
#call the head function again to view the changes
#check the length of df
#grab the text column and turn it into a list (make it into a variable "text")
#run the classifier on the text
